## Test the model

In [1]:
import numpy as np
from rouge import Rouge
import nltk
import os
from nltk import tokenize
import pandas as pd
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
rouge = Rouge()
import tensorflow as tf
import matplotlib.pyplot as plt
from summa.summarizer import summarize
from tensorflow.keras.optimizers import Adadelta
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import linear_kernel

Using TensorFlow backend.


In [2]:
def sentence_embedding(sentences):

    #Load Google pre-trained words 
    embedding_model = gensim.models.KeyedVectors.load_word2vec_format(
        'C:/Users/amitp/Downloads/extractive-document-summarization-master/word2vec/GoogleNews-vectors-negative300.bin.gz', 
        binary=True, 
        limit=50000)
    word_vectors = embedding_model.wv
    max_sen_len = 250
    #tokenize sentences
    tokenizer = Tokenizer(num_words=30000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences) # replace words with its wordid
    padded_sequences = pad_sequences(sequences,maxlen=max_sen_len)
    word_index = tokenizer.word_index
    # word embedding with 300 dimensions
    embedding_weights = {key: embedding_model[word] if word in word_vectors.vocab else
                                  np.random.uniform(-0.25, 0.25, word_vectors.vector_size)
                            for word, key in word_index.items()}
    embedding_weights[0] = np.zeros(word_vectors.vector_size)
    #Build a 3D array: 1D fnumber of sentences, 1D for the no of words and 1D for word embedding. 
    embedded_sentences = np.stack([np.stack([embedding_weights[t] for t in s]) for s in padded_sequences])
    return embedded_sentences

In [3]:
def preprocess_data_for_testing(Id,text,sumry):
    test_data = []
    for i,j in enumerate(Id):
        TES = np.ones((len(text[i]), 3), dtype=object) 
        TES[:,0] = "dummy"
        TES[:,1] = text[i]
        embedding = sentence_embedding(text[i])
        #embedding = embedding[0::2]
        test_data.append((np.array(text[i]), np.array(embedding), np.array(sumry[i])))
        print("Finished", i, "of", len(Id)+1,"sentences --", i/(len(Id)+1),"%", end='\r')
    return test_data

In [4]:
def fit_predict(test_data):
    model_summary = []
    for d,doc in enumerate(test_data): 

        sentences = doc[0]  # sentences in document
        x_test_old = doc[1] # embedding of doc
        s1 = x_test_old.shape[0] # number of sentences in document
        (s3,s4) = x_test_old[0].shape # sentence length,embedding dimension = 300

        # Data reshaping for fitting it to model.

        x_test = np.random.rand(s1,250,s4,1) # random initialization of tenst tensor
        try:
            for i in range(s1) :
                s = np.array( [ np.pad(x_test_old[i],((250-s3,0),(0,0)),mode='constant') ] )[0] # pad sentences to 250 len
                s = np.expand_dims(s, 2)
                x_test[i] = s

            predicted_scores = model1.predict(x_test, batch_size=256)
            argsorted_scores = np.argpartition(np.transpose(predicted_scores)[0], 1)

            true_summary = doc[2]
            predicted_summary = []
            summary_length = 0

            i = 0
            while i < len(sentences) and summary_length < 250: 
                sentence = sentences[argsorted_scores[i]]
                #if ( dummy_rouge( sentence , predicted_summary ) < threshold ):
                sentence = np.array([sentence])
                #print(sentence, predicted_summary)
                predicted_summary.append(sentence)
                summary_length += len(nltk.word_tokenize(sentence[0]))
                i+=1
            model_summary.append(predicted_summary)
        except:
            print(" not possible for doc number "+str(d)+" because number of sentences in the document greater than 250")
            model_summary.append(" ")
    return model_summary

In [5]:
# text cleaning = stop word removal,special characters removal , POS - Tagging   for inference logic

def getWordnetPos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:#We are igonring everything else other than four of the above 
         # tags. You can add more if you like
        return None 
#Custom function for toeknization
def myTokenizer(text):
    
    lemmatizer = WordNetLemmatizer()
    lemmas=[]
    
    for sent in nltk.sent_tokenize(text):
        #nltk return the tag from Penntreebank tagsets
        sentTag=nltk.pos_tag(nltk.word_tokenize(sent))
        sentTag=[(i.strip(),j) for i,j in sentTag if i.isalpha() and len(i)>2]
        #print (sentTag)
        for word, tag in sentTag:
            # the problem wordnet lemmatizer is that, it recognizes only
            # wordnet tags and not the PennTreebank tags. So we shall
            # first convert Penntreebank tags to Wordnet tags
            wordNetTag=getWordnetPos(tag)
            if wordNetTag is None:
                continue
            else:
                lemmas.append(lemmatizer.lemmatize(word,wordNetTag))
                
    return  lemmas


stopWords=nltk.corpus.stopwords.words('english')
stopWords+=["''", "'s", "...", "``","--","*","-"]
stopWords+=list(string.punctuation)
print("done")

done


In [20]:
# function to find similar documents serial numbers which are related to querywords

def query_and_op(query_string,number_of_doc, tf_idf):
    """
    Input: 
    
    - query_string - input all query words in a single string.
    - number_of_doc - number of similar documents to extract
    - tf_idf - tf_idf scores saved in a dataframe format where df indexes are vocabulary and columns are docs and cell 
               values are the tf-idf scores
    
    output: index of revelent documents or document number best describe the query string words.
    
    """
    query = query_string
    query_tokens = myTokenizer(query)
    
    filtered = [] # term-doc
    for i in query_tokens:
        #print(i)
        i = i.lower().strip()
        try:
            i = tf_doc_df.loc[i].values
            filtered.append(list(i))

        except:
            filtered.append([0.0]*tf_doc_df.shape[0])
            
    doc_term = np.array(filtered).T                   #doc*term
    #doc_term.shape                                   #(number of doc * number of term)
    tf_idf_score = np.sum(doc_term,axis=1)
    most_revelent_doc = np.argsort(-tf_idf_score)[0]  # sort document according to highest similarity score.
    
    doc_similarity = linear_kernel(doc_term[most_revelent_doc:most_revelent_doc+1],doc_term)
    simlar_doc_query_object = np.argsort(-doc_similarity[0])
    test_doc = simlar_doc_query_object[:number_of_doc] # top 20 documents describing the query words
    
    return test_doc

In [7]:
# function to extract summary:

def extract_summary(doc_to_query,similarity_indx, title = None,content = None,summary = None,test_rank = None):
    test = doc_to_query.loc[test_doc]
    test = test.reset_index()
    test = test.drop('index',axis = 1)
    test = test.reset_index()
    test = test.drop('index',axis = 1)
    test = test.reset_index()
    
    test_data = preprocess_data_for_testing(test.index,test.content,test.Summary_regulation)
    prediction = fit_predict(test_data)
    
    for i in range(len(test_data)):
        if title:
            print("Title :",query_documents.loc[similarity_indx[i]].longtitle,"\n")
        if content:
            print("Content :",query_documents.loc[similarity_indx[i]].content,"\n")
        if summary:
            print("summary :",prediction[i][0][0])
        if test_rank:
            print("test_rank :",query_documents.loc[similarity_indx[i]].Summary_regulation,"\n")
            
        print("\n**************************\n")
    

    

In [9]:
model1 = tf.keras.models.load_model('best_model_cnn.h5')
model1.compile(loss='binary_crossentropy',optimizer=Adadelta(),metrics=['mae'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 1, 50)        45050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 1, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 603       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 45,657
Trainable params: 45,657
Non-trainable params: 0
__________________________________________________

### use tf-idf to extract documents and summary.

In [10]:
query_documents = pd.read_csv("C:/Users/amitp/OneDrive/Ryerson-DS/MRP/regulation.csv",encoding='utf-8')

In [11]:
indx =[]
content = []
for i,j in enumerate(query_documents.content):
    j = nltk.sent_tokenize(j)
    if "Interpretation." in j:
        ind = j.index("Interpretation.")
        indx.append(i)
        content.append(j[ind+1:])
           

In [12]:
query_documents = query_documents.loc[indx]
query_documents["content"]= content
query_documents = query_documents.reset_index(drop= True)
query_documents = query_documents.drop(['consolidationyear','registrationyear','regnalyears', 'shorttitle', 'xrefinternal', 'xrefxternal'], axis=1)


In [13]:
# creating summary by page- rank as we do not have it for regulation data.
# converting string summary into list of sentences
Summary_regulation = []

for i in query_documents.content:
    s  = summarize(" ".join(i), ratio=0.2)
    Summary_regulation.append(s)
cont = []    

for i in Summary_regulation:
    i = nltk.sent_tokenize(i)
    cont.append(i)
query_documents['Summary_regulation']= cont
del Summary_regulation 

In [14]:
# create term-document matrix of text corpus: if not already created uncomment this section and genetate these files


#documents = pd.read_csv("C:/Users/amitp/OneDrive/Ryerson-DS/MRP/regulation.csv",encoding='utf-8')
#documents = documents.loc[indx]
#text_corpus = documents.content.values

#vectorizer = TfidfVectorizer(max_features=5000, max_df=1.0,tokenizer=myTokenizer, stop_words=stopWords)

#tf_doc = vectorizer.fit_transform(text_corpus)
#tf_idf_features = vectorizer.get_feature_names()

#pickle.dump(tf_doc, open("tf_idf_vectorizer.pickle", "wb"))
#pickle.dump(tf_idf_features, open("tf_idf_features.pickle", "wb"))

   #### steps to initiate   document searching:

In [15]:
# Import if-idf scores . If not already processed then uncomment the above cell get the fils.

vectorizer = pickle.load(open("tf_idf_vectorizer.pickle", "rb"))
features = pickle.load(open("tf_idf_features.pickle", "rb"))
tf_doc_df = pd.DataFrame(vectorizer.toarray().T,index = features )

In [22]:
#   give the input words to query stings which will extract the documents numbers.
query_string =  "airport security"
number_of_doc = 2
tf_idf = tf_doc_df
test_doc = query_and_op( query_string, number_of_doc, tf_doc_df )
# run the function extract summary as per required setting to get final output
doc_to_query = query_documents
similarity_indx = test_doc
title = True
content = False
summary = True
test_rank = False
extract_summary(doc_to_query,similarity_indx, title,content,summary,test_rank)

        

c:\users\amitp\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Title : Regulations Respecting Reservations for Airport Apron and Terminal Facilities for Passenger Charter Flights 

summary : In these Regulations, air carrier  means any person who operates a commercial air service; ( transporteur aérien ) airport manager  means the person in charge of an airport or the authorized representative of that person; ( directeur d’aéroport ) applicant  means an air carrier who applies for a reservation referred to in section 4; ( requérant ) regular charter flight  means a passenger charter flight that is operated by an air carrier on a program basis and for which schedules are provided six months in advance in respect of international flights and three months in advance in respect of domestic flights to the airport manager of each airport listed in the schedule that the air carrier uses for the purposes of the flight, and the apron and terminal facilities of each airport are reserved through established scheduling procedures; ( vol d’affrètement régulier